To begin with, here's the paper on which we're going to base our generative adversarial network (GAN):  
https://www.mdpi.com/2227-7080/12/10/186

Start by downloading the dataset on Kaggle


#!/bin/bash  
curl -L -o ./dataset/creditcardfraud.zip https://www.kaggle.com/api/v1/datasets/download/mlg-ulb/creditcardfraud

unzip ./dataset/creditcardfraud.zip -d ./dataset/  
rm ./dataset/creditcardfraud.zip

In [5]:
# Regular imports
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [17]:
df = pd.read_csv("./dataset/creditcard.csv")

In [18]:
df.head(1)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.5516,-0.617801,-0.99139,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0


In [ ]:
# Check for null values in the dataset
df.isnull().sum(axis=0).sum()

np.int64(0)

In [19]:
# Ensure that tensorflow use a GPU
import tensorflow as tf
len(tf.config.experimental.list_physical_devices('GPU'))

1

In [11]:
import numpy as np
from tensorflow.keras import layers, models
from keras import Model
from keras.layers import Conv2D, PReLU,BatchNormalization, Flatten, GRU
from keras.layers import UpSampling2D, LeakyReLU, Dense, Input, add
from tqdm import tqdm
from numpy.random import randint
from keras.models import load_model

In [6]:
def build_generator(input_dim, output_dim):
    """
    Build a generator model with dense layers.
    
    :param input_dim: Dimensionality of the noise vector (e.g., 100)
    :param output_dim: Dimensionality of the output (e.g., 30 columns for your dataset)
    :return: A Keras Model representing the generator.
    """
    # Input
    input_layer = Input(shape = (input_dim,))

    # Hidden layers
    layer = Dense(128, activation="relu")(input_layer)
    layer = Dense(256, activation="relu")(layer)
    layer = Dense(512, activation="relu")(layer)

    # Output
    output_layer = Dense(output_dim, activation="linear")(layer)

    return Model(input_layer, output_layer)

In [7]:
def build_discriminator(input_dim):
    """
    Build a discriminator model using GRU layers.
    
    :param input_dim: Dimensionality of the input (e.g., 30 columns in your dataset)
    :return: A Keras Model representing the discriminator.
    """
    # Input
    input_layer = Input(shape = (input_dim, 1))

    # Hidden layers
    layer = GRU(64, return_sequences=True, activation="relu")(input_layer)
    layer = GRU(128, activation="relu")(layer)

    layer = Dense(64, activation="relu")(layer)

    # Output
    output_layer = Dense(1, activation="sigmoid")(layer)

    return Model(input_layer, output_layer)

In [13]:
def build_combined(generator, discriminator, noise_shape):

    # Input
    input_layer = Input(shape = (noise_shape,))

    # Use discriminator to train generator
    fake_sample = generator(input_layer)

    validity = discriminator(fake_sample)

    return Model(input_layer, validity)

In [46]:
generator = build_generator(100, 30)
generator.compile(loss = "binary_crossentropy", optimizer = "adam")
discriminator = build_discriminator(30)
discriminator.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])
combined = build_combined(generator, discriminator, 100)
combined.compile(loss = "binary_crossentropy", optimizer = "adam")

In [ ]:
def train(real_samples, epochs = 10, batch_size = 64):

    noise_batches = []
    real_sample_batches = []
    
    for nb_batch in range(int(real_samples.shape[0] / batch_size)) :
        
        start_idx = nb_batch * batch_size
        end_idx = start_idx + batch_size
        noise_batches.append(np.random.uniform(0, 1, (batch_size, 100)))
        real_sample_batches.append(real_samples[start_idx : end_idx])

    for e in tqdm(range(epochs)):

        for batch in range(len(real_sample_batches)):

            _real_samples = real_sample_batches[batch]

            noises = noise_batches[batch]
    
            fake_samples = generator(noises)

            discriminator.trainable = True
            fake_loss = discriminator.train_on_batch(fake_samples, np.zeros((batch_size, 1)))
            real_loss = discriminator.train_on_batch(_real_samples, np.ones((batch_size, 1)))

            temp_d_loss = np.add(fake_loss, real_loss) * 0.5

            discriminator.trainable = False

            temp_g_loss = combined.train_on_batch(noises, np.ones((batch_size, 1)))

            g_loss.append(temp_g_loss)
            d_loss.append(temp_d_loss)
            
        g_loss = np.array(g_loss)
        d_loss = np.array(d_loss)
        
        g_loss = g_loss.mean(axis = 0)
        d_loss = d_loss.mean(axis = 0)
        
        print(f"Epochs : {e + 1} | generator loss : {g_loss} | discriminator loss : {d_loss}")

In [52]:
X = df.drop("Class", axis=1)
y = df["Class"]

In [53]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [54]:
train(X_scaled)

  0%|          | 0/10 [00:00<?, ?it/s]

(64, 30)


  0%|          | 0/10 [00:01<?, ?it/s]


AttributeError: 'NoneType' object has no attribute 'update_state'